# MPRA regression with K-fold cross validation

### Environment 
The next chunk contains the commands necessary to install the environment required to run this jupyter notebook
Skip this chunk if the installation was previously done

In [6]:
%%bash
conda create --name tf_MPRA python=3.9.7
conda activate tf_MPRA
pip install tensorflow[and-cuda]
conda install -c anaconda ipykernel 
conda install -c anaconda pandas
conda install -c anaconda numpy
conda install -c anaconda scikit-learn 
conda install -c conda-forge matplotlib

# After installation if you are using VSCODE to run the notebook you have to close it and re-open


Remove existing environment (y/[n])? Invalid choice: conda activate tf_mpra
Remove existing environment (y/[n])? Invalid choice: pip install tensorflow[and-cuda]
Remove existing environment (y/[n])? Invalid choice: conda install -c anaconda ipykernel
Remove existing environment (y/[n])? Invalid choice: conda install -c anaconda pandas
Remove existing environment (y/[n])? Invalid choice: conda install -c anaconda numpy
Remove existing environment (y/[n])? Invalid choice: conda install -c anaconda scikit-learn
Remove existing environment (y/[n])? Invalid choice: conda install -c conda-forge matplotlib
Remove existing environment (y/[n])? 



CondaSystemExit: Exiting.



### Library imports


In [7]:
import os 
import getopt
import sys

import numpy as np
import h5py
import pickle
import random
import copy
import pandas as pd
import math 

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Layer, Lambda, concatenate, Bidirectional, Dense, Dropout, Flatten, Conv1D,BatchNormalization,  MaxPooling1D, Bidirectional, GRU, TimeDistributed
from tensorflow.keras.layers import Normalization
from tensorflow.keras.initializers import Constant
import tensorflow as tf
from tensorflow import keras


### Input ingestion

Here we define the methods to read and ingest data and we initialize the random seed.

Since we are processing the entire sequence the vocabulary is comprised of upper case nucleotides


In [8]:
np.random.seed(1337) # for reproducibility

# Lower case vocabulary
vocab = ["A", "G", "C", "T"]

# These are the defaults of the data reader method 
# (each column in the ingested csv must be initialized with the right data type, otherwise the data ingestion fails )
indices = tf.range(len(vocab), dtype = tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab,indices)
table = tf.lookup.StaticVocabularyTable(table_init, 1)
defs = [0.] * 1 + [tf.constant([], dtype = "string")]

# Nadav dataset

def data_reader(file, batch_size=100, n_parse_threads=4):
    """Method for reading the data in an optimized way, can be used inside model.fit()
    
    Args:
        file (_type_): path to csv file
        batch_size (int, optional): _description_. Defaults to 100.
        n_parse_threads (int, optional): _description_. Defaults to 4.

    Returns:
        dataset.batch: batch dataset object 
    """
    dataset = tf.data.TextLineDataset(file).skip(1)
    dataset=dataset.map(preprocess, num_parallel_calls = n_parse_threads)
    return dataset.batch(batch_size).prefetch(1)

def preprocess(record):
    """Preprocessing method of a dataset object, one-hot-encodes the data

    Args:
        record (_type_): _description_

    Returns:
        X (2D np.array): one-hot-encoded input sequence
        Y (1D np.array): MPRA measurements

    """
    fields = tf.io.decode_csv(record, record_defaults=defs)
    chars = tf.strings.bytes_split(fields[1])
    chars_indeces = table.lookup(chars)
    X = tf.one_hot(chars_indeces, depth = len(vocab))
    Y = fields[0]
    return X,Y

### k-fold cross validation split
Here we take the initial csv file and we split it in 3 partitions k times

It is possible to randomize the sequences and augment, since the masking of the model motifs was a better choice
for understanding the background this strategy is here commented out and not used


In [9]:
# CROSS VALIDATION (10 fold)
import pandas as pd
from sklearn.model_selection import train_test_split, KFold

# Split the data in three partitions
whole_data = pd.read_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/LibA_wide_pivot_state3.csv")

kf = KFold(n_splits = 10, shuffle = True, random_state = 2008)

o=1
# For each fold we split again to get the third partition
for i in kf.split(whole_data):
    # Get train/test split and upper case all nucleotides
    train = whole_data.iloc[i[0]]
    train["seq"] = train['seq'].str.upper() 
    
    test =  whole_data.iloc[i[1]]
    test["seq"] = test['seq'].str.upper() 

    train, validation = train_test_split(train, test_size=0.11, random_state=42)
    
    train.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(o)+"_LibA_wide_pivot_state3_train.csv", index=False)
    test.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(o)+"_LibA_wide_pivot_state3_test.csv", index=False)
    validation.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(o)+"_LibA_wide_pivot_state3_validation.csv", index=False)
    o+=1

train

/tmp/ipykernel_1740449/1817506747.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["seq"] = train['seq'].str.upper()
/tmp/ipykernel_1740449/1817506747.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["seq"] = test['seq'].str.upper()


,State_3E,seq
5786,0.053119,AGGACCGGATCAACTGGTGCTGCCGTCACAACGCACTGTGCTTGTT...
5873,-0.077200,AGGACCGGATCAACTGTACAGTTATAAGGTAATCTTGTTCGATAAA...
3331,0.011841,AGGACCGGATCAACTCGACCGGGGTCACCAGGATATTATCAGATGG...
539,-0.103969,AGGACCGGATCAACTAATTGATATTGGCGCGTGTATCCCGAATTTC...
1122,0.036170,AGGACCGGATCAACTACTTGTGTTCAGGCACGATTTCTATCGTCGG...
...,...,...
5792,0.151705,AGGACCGGATCAACTGGTGGTAGCCACCTAGGGATAGCATAGGAGG...
5980,-0.000154,AGGACCGGATCAACTGTCAGAACTGCTTTGCCGACCGCGCTTTCTT...
955,0.039057,AGGACCGGATCAACTACGTAGTGCTTTATACCATCGAACCACTGCG...
8448,0.091301,AGGACCGGATCAACTTTTGTGTAACGCTGTACGGGGCTGGAGATTT...


### Deep Learning model

Here we run the model which is based on this paper : 

https://doi.org/10.1101/2023.03.05.531189

I have added a Normalization layer parametrized with two parameters. 

In [10]:

df_test_10folds  = pd.DataFrame(columns=['State_3E', "seq", "prediction"])
corr_list = []

# We define a custom normalization layer to then compile on the model
class CustomNormalization(Layer):
    """Custom normalization layer that normalizes the output of the neural network"""
    def __init__(self, **kwargs):
        super(CustomNormalization, self).__init__(**kwargs)
        
    def build(self, input_shape):
        # Add trainable variables for mean and standard deviation
        self.mean = self.add_weight("mean", shape=(1,), initializer="zeros", trainable=True)
        self.stddev = self.add_weight("stddev", shape=(1,), initializer="ones", trainable=True)
        super(CustomNormalization, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs):
        # Normalize the inputs using the learned mean and standard deviation
        return (inputs - self.mean) / (self.stddev + 1e-8)

# We define the method to compute the pearson correlation between prediction and ground truth
def pearson_correlation(x, y):
    """Computes Pearson Correlation between x and y
    Args:
        x (np.array): vector of predictions values
        y (np.array): vector of ground truth values

    Returns:
        (float): pearson correlation
    """
    n = len(x)
    
    # Calculate the mean of x and y
    mean_x = sum(x) / n
    mean_y = sum(y) / n
    
    # Calculate the numerator and denominators of the correlation coefficient
    numerator = sum((xi - mean_x) * (yi - mean_y) for xi, yi in zip(x, y))
    denominator_x = math.sqrt(sum((xi - mean_x) ** 2 for xi in x))
    denominator_y = math.sqrt(sum((yi - mean_y) ** 2 for yi in y))
    
    # Calculate the correlation coefficient
    correlation = numerator / (denominator_x * denominator_y)
    return correlation

import matplotlib.pyplot as plt
%matplotlib inline
# Define plotting function of loss
def create_plots(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.clf()


NameError: name 'Layer' is not defined

#### Model training
Here we iterate through the folds and train the model

In [ ]:

for i in range(1,11):
    
    # Define inputs
    input_path_train = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(i)+"_LibA_wide_pivot_state3_train.csv"
    input_path_valid = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(i)+"_LibA_wide_pivot_state3_validation.csv"
    input_path_test = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(i)+"_LibA_wide_pivot_state3_test.csv"
    
    # Read test data to then predict
    df_test = pd.read_csv(input_path_test)

    # Get first item of the dataset to get the shape of the input data
    for element in data_reader(input_path_train):
        input_shape = element[0].shape
    
    # Define and compile model
    inputs = Input(shape=(input_shape[1],input_shape[2]), name="inputs")
    layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
    layer = Dropout(0.3)(layer)
    layer = BatchNormalization()(layer)
    layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
    layer = BatchNormalization()(layer)
    layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
    layer = Dropout(0.3)(layer)
    layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
    layer = BatchNormalization()(layer)
    layer = Dropout(0.3)(layer)
    layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
    layer = BatchNormalization()(layer)
    layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
    layer = Dropout(0.3)(layer)
    layer = Flatten()(layer)
    layer = Dense(300, activation='sigmoid')(layer)  # 300
    layer = Dropout(0.3)(layer)
    layer = Dense(200, activation='sigmoid')(layer)  # 300
    predictions = Dense(1, activation='linear')(layer)
    norm_predictions = CustomNormalization()(predictions)  # Assuming "predictions" is your existing output

    model = Model(inputs=inputs, outputs=norm_predictions)
    model.summary()

    # Compile model
    model.compile(optimizer="adam",
                loss="mean_squared_error",
                metrics=["mse", "mae", "mape"],
                )
    # Run model
    history=model.fit(data_reader(input_path_train, batch_size=200),
                            epochs=20,
                            validation_data=data_reader(input_path_valid,batch_size=100),
                            callbacks=None,
                            verbose=1)
    
    #After training we save the model weights to then run the contribution scores
    model_path = '/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/Model_CV'+str(i)+"_LibA_wide_pivot_state3.h5"
    model.save_weights(model_path, save_format='h5')
    
    # We predict the test data
    predicted = model.predict(data_reader(input_path_test,
                                                batch_size=100))

    # We fill the dataframe with predictions and fold annotation
    test_data = data_reader(input_path_test,batch_size=100)
    test_tensor = X = np.empty(shape=[0,1])
    for batch in test_data:
        test_tensor = np.append(test_tensor, batch[1])

    # Append fold to previous folds
    df_test["prediction"] = predicted
    df_test["fold"] = str(i)
    df_test_10folds = pd.concat([df_test_10folds, df_test], ignore_index=True)    

    # Append correlation coefficient and append to previous        
    corr_coefficient = pearson_correlation(predicted.flatten(), test_tensor)
    corr_list.append(corr_coefficient)
    break

df_test_10folds.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/LibA_wide_pivot_state3_test_predicted_cv10fold.csv", index=False)
print(corr_list)
df_test_10folds

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 262, 4)]          0         
                                                                 
 conv1 (Conv1D)              (None, 256, 250)          7250      
                                                                 
 dropout_65 (Dropout)        (None, 256, 250)          0         
                                                                 
 batch_normalization_52 (Ba  (None, 256, 250)          1000      
 tchNormalization)                                               
                                                                 
 conv2 (Conv1D)              (None, 249, 250)          500250    
                                                                 
 batch_normalization_53 (Ba  (None, 249, 250)          1000      
 tchNormalization)                                        

2023-11-08 15:34:05.900902: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.31MiB (rounded to 2420224)requested by op model_12/dropout_68/dropout/GreaterEqual
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-11-08 15:34:05.901038: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-11-08 15:34:05.901065: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 436, Chunks in use: 434. 109.0KiB allocated for chunks. 108.5KiB in use in bin. 1.9KiB client-requested in use in bin.
2023-11-08 15:34:05.901083: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 147, Chunks in use: 146. 75.0KiB allocated for chunks. 74.5KiB in use in bin. 57.0KiB client-requested in use in bin.
2023-11-08 15:34:0

ResourceExhaustedError: Graph execution error:

Detected at node model_12/dropout_68/dropout/GreaterEqual defined at (most recent call last):
  File "/home/felix/anaconda3/envs/tf_MPRA/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/home/felix/anaconda3/envs/tf_MPRA/lib/python3.9/runpy.py", line 87, in _run_code

  File "/home/felix/anaconda3/envs/tf_MPRA/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/felix/anaconda3/envs/tf_MPRA/lib/python3.9/site-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/home/felix/anaconda3/envs/tf_MPRA/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/home/felix/anaconda3/envs/tf_MPRA/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/home/felix/anaconda3/envs/tf_MPRA/lib/python3.9/asyncio/base_events.py", line 596, in run_forever

  File "/home/felix/anaconda3/envs/tf_MPRA/lib/python3.9/asyncio/bas

In [ ]:

create_plots(history)

# We now compute an ensemble approach we compute for each fold the model 10 times


In [ ]:
df_test_10folds  = pd.DataFrame()
corr_list = []

for i in range(1,11):
    
    input_path_train = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(i)+"_LibA_wide_pivot_state3_train.csv"
    input_path_valid = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(i)+"_LibA_wide_pivot_state3_validation.csv"
    input_path_test = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(i)+"_LibA_wide_pivot_state3_test.csv"
   
    df_test = pd.read_csv(input_path_test)
    df_test["fold"] = str(i)
    corr_per_iteration = []
    # Get first item of the dataset to get the shape of the input data
    for element in data_reader(input_path_train):
        input_shape = element[0].shape
        
    for iteration in range(1,11):
        inputs = Input(shape=(input_shape[1],input_shape[2]), name="inputs")
        layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
        layer = Dropout(0.3)(layer)
        layer = BatchNormalization()(layer)
        layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
        layer = Dropout(0.3)(layer)
        layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
        layer = BatchNormalization()(layer)
        layer = Dropout(0.3)(layer)
        layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
        layer = Dropout(0.3)(layer)
        layer = Flatten()(layer)
        layer = Dense(300, activation='sigmoid')(layer)  # 300
        layer = Dropout(0.3)(layer)
        layer = Dense(200, activation='sigmoid')(layer)  # 300
        predictions = Dense(1, activation='linear')(layer)

        model = Model(inputs=inputs, outputs=predictions)
        model.compile(optimizer="adam",
                    loss="mean_squared_error",
                    metrics=["mse", "mae", "mape"],
                    )

        history=model.fit(data_reader(input_path_train, batch_size=100),
                                epochs=20,
                                validation_data=data_reader(input_path_valid,batch_size=100),
                                callbacks=None,
                                verbose=2)

        predicted = model.predict(data_reader(input_path_test,
                                                    batch_size=100))

        test_data = data_reader(input_path_test,batch_size=100)
        test_tensor = X = np.empty(shape=[0,1])
        for batch in test_data:
            test_tensor = np.append(test_tensor, batch[1])

        df_test["prediction_iteration_"+str(iteration)] = predicted
        
                
        corr_coefficient = pearson_correlation(predicted.flatten(), test_tensor)
        corr_per_iteration.append(corr_coefficient)
    
    df_test_10folds = df_test_10folds.append(df_test, ignore_index=True)
    print(df_test_10folds)
        
    corr_ensemble = np.mean(corr_per_iteration)
    corr_list.append(corr_ensemble)

df_test_10folds.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/LibA_wide_pivot_state3_test_predicted_cv10fold_ensemble.csv", index=False)